# Documentation

# Sample

In [1]:
import sys 
!{sys.executable} -m pip install nltk 
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install unidecode

You should consider upgrading via the '/Users/pierre.krzisch/.pyenv/versions/3.10.2/envs/jupyter-dev/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/pierre.krzisch/.pyenv/versions/3.10.2/envs/jupyter-dev/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/pierre.krzisch/.pyenv/versions/3.10.2/envs/jupyter-dev/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/pierre.krzisch/.pyenv/versions/3.10.2/envs/jupyter-dev/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/pierre.krzisch/.pyenv/versions/3.10.2/envs/jupyter-dev/bin/python3.10 -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from unidecode import unidecode

nltk.download('stopwords')

products_data = pd.read_csv("produtos.csv", delimiter=';', encoding='utf-8')

# concatenando as colunas nome e descricao
products_data['informacao'] = products_data['nome'] + products_data['descricao']
# excluindo linhas com valor de informacao ou categoria NaN
products_data.dropna(subset=['informacao', 'categoria'], inplace=True)
products_data.drop(columns=['nome', 'descricao'], inplace=True)

stop_words=set(stopwords.words("portuguese"))
# transforma a string em caixa baixa e remove stopwords
products_data['sem_stopwords'] = products_data['informacao'].str.lower().apply(lambda x: ' '.join([unidecode(word) for word in x.split() if word not in (stop_words)]))

tokenizer = nltk.RegexpTokenizer(r"\w+")
products_data['tokens'] = products_data['sem_stopwords'].apply(tokenizer.tokenize) # aplica o regex tokenizer
products_data.drop(columns=['sem_stopwords','informacao'],inplace=True) # Exclui as colunas antigas

products_data["strings"]= products_data["tokens"].str.join(" ") # reunindo cada elemento da lista
products_data.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pierre.krzisch/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,categoria,tokens,strings
0,livro,"[hobbit, 7a, ed, 2013, produto, novobilbo, bol...",hobbit 7a ed 2013 produto novobilbo bolseiro h...
1,livro,"[livro, it, coisa, stephen, king, produto, nov...",livro it coisa stephen king produto novodurant...
2,livro,"[box, cronicas, gelo, fogo, pocket, 5, livros,...",box cronicas gelo fogo pocket 5 livros produto...
3,livro,"[box, harry, potter, produto, novo, fisico, se...",box harry potter produto novo fisico serie har...
4,livro,"[livro, origem, dan, brown, produto, novode, o...",livro origem dan brown produto novode onde vie...


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

X_train,X_test,y_train,y_test = train_test_split( # Separação dos dados para teste e treino
    products_data["strings"], 
    products_data["categoria"], 
    test_size = 0.2, 
    random_state = 10
)

pipe = Pipeline([('vetorizador', CountVectorizer()), ("classificador", MultinomialNB())])

In [4]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('vetorizador', CountVectorizer()),
                ('classificador', MultinomialNB())])

In [5]:
from sklearn.metrics import accuracy_score
from sklearn import metrics

y_prediction = pipe.predict(X_test)
accuracy = accuracy_score(y_prediction, y_test)

In [6]:
input_message = ["Figura Transformers Prime War Deluxe - E9687 - Hasbro",  "Senhor dos aneis", "Senhor dos anéis"]

In [8]:
import joblib

final_prediction = pipe.predict(input_message)
print("Predicted values:")
print(",".join(final_prediction))

# joblib.dump(pipe, "classificador-produtos-1.0.joblib")
joblib.dump(pipe, "classificador-produtos-1.1.joblib")

Predicted values:
brinquedo,livro,livro


['classificador-produtos-1.1.joblib']